In [ ]:
#combined bradford and airedale queries for injurious falls 
create or replace table `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_injurious_falls_v3` as (
with fall_aire as  
(
with episode_ch_cohort_2015 as (
select 
person_id, 
tbl_SUS_Airedale_EC_BackwardCompatible_20150401_to_20190131_start_date as episode_start, 
tbl_SUS_Airedale_EC_BackwardCompatible_20150401_to_20190131_end_date as episode_end, 
1 as episode_sequence, 
rtrim(diag1,'X') as diagnosis_1, 
rtrim(diag2,'X') as diagnosis_2,
rtrim(diag3,'X') as diagnosis_3, 
from `yhcr-prd-bradfor-bia-core.CB_FDM_Airedale.tbl_SUS_Airedale_EC_BackwardCompatible_20150401_to_20190131`
where person_id in(select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` )),

episode_ch_cohort_2019 as (
  select 
person_id, 
tbl_SUS_Airedale_APC_20190201_to_20200630_start_date as episode_start, 
tbl_SUS_Airedale_APC_20190201_to_20200630_end_date as episode_end, 
cast(ep_no as INT64) as episode_sequence, 
rtrim(diag1,'X') as diagnosis_1, 
rtrim(diag2,'X') as diagnosis_2,
rtrim(diag3,'X') as diagnosis_3, 
from `yhcr-prd-bradfor-bia-core.CB_FDM_Airedale.tbl_SUS_Airedale_APC_20190201_to_20200630`
where person_id in(select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`)
),

episode_ch_cohort_2020 as (
  select 
person_id, 
tbl_SUS_Airedale_APC_20200701_to_20220710_mrg_start_date as episode_start, 
tbl_SUS_Airedale_APC_20200701_to_20220710_mrg_end_date as episode_end, 
cast(ep_no as INT64) as episode_sequence, 
rtrim(diag1,'X') as diagnosis_1, 
rtrim(diag2,'X') as diagnosis_2,
rtrim(diag3,'X') as diagnosis_3, 
from `yhcr-prd-bradfor-bia-core.CB_FDM_Airedale.tbl_SUS_Airedale_APC_20200701_to_20220710_mrg`
where person_id in(select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`)
),
#for some reason the airedale hospital records are split into prea nd post 2019 so i had to combine the two tables toether to ensure no episodes were missed 

episode_ch_cohort_combined as (
  select
  * 
  from episode_ch_cohort_2015

  union all

  select
  * 
  from episode_ch_cohort_2019

  union all

  select
  *
  from episode_ch_cohort_2020
),


#joins care home entry date from the master cohort table 
episode_entrydate_join as (
  select 
  e_c.*,
  cast(a.first_episodestartdate as datetime) first_episodestartdate
  from episode_ch_cohort_combined as e_c
  left join `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` a
  on e_c.person_id = a.person_id),
#filters out episodes which began after the admission date to a care home
episode_ch_filtered_ae as (
  select
e_e_j.*
  from episode_entrydate_join e_e_j
  where (episode_start > first_episodestartdate) AND (datetime_diff(episode_start,first_episodestartdate, day) < 366)
),
#filter for the first episode in a spell as to avoid inpatient falls being inadvertently captured. 
episode_ch_final_ae as (
select 
person_id, 
first_episodestartdate,
substr(diagnosis_1,1,3) as diagnosis_1_substr,
substr(diagnosis_2,1,3) as diagnosis_2_substr,
substr(diagnosis_3,1,3) as diagnosis_3_substr,
episode_start,
episode_end,   
from episode_ch_filtered_ae
where episode_sequence = 1
order by person_id),
#up to this point reprents the code to identify and clean alll seocndary inpatient episodes for indiivudals in the connected bradford cohort in the 12 months after admission,the next line of code will identify admission related to falls, will then union this with a table for falls in AE

fall_ch as (
select
*,
#this line ranks episodes according to the date they occurred for each person, this way we can identify the most recent hospital attendance following admission 
row_number() over (partition by person_id order by episode_start) as admission_seq
from episode_ch_final_ae 
where episode_ch_final_ae.diagnosis_1_substr in(select Code from `yhcr-prd-bradfor-bia-core.CB_2172.falls__fragilityfracture_icd10`) OR diagnosis_2_substr in(select Code from `yhcr-prd-bradfor-bia-core.CB_2172.falls__fragilityfracture_icd10`) OR diagnosis_3_substr in(select Code from `yhcr-prd-bradfor-bia-core.CB_2172.falls__fragilityfracture_icd10`)
order by person_id)
#this then filters for the first hopsital spell with a fall coded in any of the diagnosis codes. it is theoretically possible that a patient had an inpatient fall which was coded in the co-moribidities rather than the primary reason for presntation, but it is assumed the margin of error is relatively small.

select
*
from fall_ch
where admission_seq = 1),

fall_bradford as (
  with episode_ch_cohort_ae as (
select 
person_id, 
tbl_ae_start_date, 
tbl_ae_end_date, 
rtrim(diagnosis_1,'X') as diagnosis_1, 
rtrim(diagnosis_2,'X') as diagnosis_2,
rtrim(diagnosis_3,'X') as diagnosis_3, 
from `yhcr-prd-bradfor-bia-core.CB_FDM_BradfordRoyalInfirmary.tbl_ae`
where person_id in(select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` )),

#joins care home entry date from the master cohort table 
episode_entrydate_join_ae as (
  select 
  e_c.*,
  cast(a.first_episodestartdate as datetime) first_episodestartdate
  from episode_ch_cohort_ae as e_c
  left join `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` a
  on e_c.person_id = a.person_id),
#filters out episodes which began after the admission date to a care home
episode_ch_filtered_ae as (
  select
e_e_j.*
  from episode_entrydate_join_ae e_e_j
  where (tbl_ae_start_date > first_episodestartdate) AND (datetime_diff(tbl_ae_start_date, first_episodestartdate, day) < 366 )
),
#filter for the first episode in a spell and extract the first three characters from each icd10 diagnosis code (not interested in the fourth character code for falls code)
episode_ch_final_ae as (
select 
person_id, 
first_episodestartdate,
diagnosis_1,
diagnosis_2,
diagnosis_3,
tbl_ae_start_date,
tbl_ae_end_date,   
substr(diagnosis_1,1,3) as diagnosis_1_substr,
substr(diagnosis_2,1,3) as diagnosis_2_substr,
substr(diagnosis_3,1,3) as diagnosis_3_substr,
from episode_ch_filtered_ae
order by person_id),
#up to this point reprents the code to identify and clean all seocndary inpatient episodes for indiivudals in the connected bradford cohort in the 12 months after admission,the next line of code will identify admission related to falls, will then union this with a table for falls in AE. admissions related to falls was defined as any admission with a SNOMED OR ICD10 code available. 

fall_ch_ae as (
select
*,
row_number() over (partition by person_id order by tbl_ae_start_date) as admission_seq
from episode_ch_final_ae 
where episode_ch_final_ae.diagnosis_1_substr in(select Code from `yhcr-prd-bradfor-bia-core.CB_2172.falls__fragilityfracture_icd10`) OR episode_ch_final_ae.diagnosis_2_substr in(select Code from `yhcr-prd-bradfor-bia-core.CB_2172.falls__fragilityfracture_icd10`) OR diagnosis_3_substr in(select Code from `yhcr-prd-bradfor-bia-core.CB_2172.falls__fragilityfracture_icd10`) or diagnosis_1_substr in(select cast(code as strAing) from `yhcr-prd-bradfor-bia-core.CB_2172.falls_snomed` ) or diagnosis_2_substr in(select cast(code as string) from `yhcr-prd-bradfor-bia-core.CB_2172.falls_snomed` ) or diagnosis_3_substr in(select cast(code as string) from `yhcr-prd-bradfor-bia-core.CB_2172.falls_snomed` )
order by person_id),
#this then filters for the first hopsital spell with a fall coded in any of the diagnosis codes. it is theoretically possible that a patient had an inpatient fall which was coded in the co-moribdities rather than the primary reason for presntation, but it is assumed the margin of error is relatively small.

fall_ae_final as(
select
*,
from fall_ch_ae
where admission_seq = 1
order by person_id),

#then we take inpatient icd10 diagnosis for BRI 
  episode_ch_cohort as (
 
select 
person_id, 
tbl_episode_start_date, 
tbl_episode_end_date, 
episode_number, 
spell_number, 
cast(episode_sequence as INT64) as episode_sequence, 
current_episode, 
last_episode_in_spell, 
diagnoses, 
rtrim(diagnosis_1,'X') as diagnosis_1, 
rtrim(diagnosis_2,'X') as diagnosis_2,
rtrim(diagnosis_3,'X') as diagnosis_3, 
from `yhcr-prd-bradfor-bia-core.CB_FDM_BradfordRoyalInfirmary.tbl_episode`
where person_id in(select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` )),

#joins care home entry date from the master cohort table 
episode_entrydate_join as (
  select 
  e_c.*,
  cast(a.first_episodestartdate as datetime) first_episodestartdate
  from episode_ch_cohort as e_c
  left join `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` a
  on e_c.person_id = a.person_id),
#filters out episodes which began after the admission date to a care home
episode_ch_filtered as (
  select
e_e_j.*
  from episode_entrydate_join e_e_j
  where (tbl_episode_start_date > first_episodestartdate) AND (datetime_diff(tbl_episode_start_date, first_episodestartdate, day) < 366 )
),
#filter for the first episode in a spell and extract the first three characters from each icd10 diagnosis code (not interested in the fourth character code for falls code)
episode_ch_final as (
select 
person_id, 
first_episodestartdate,
diagnosis_1,
diagnosis_2,
diagnosis_3,
tbl_episode_start_date,
tbl_episode_end_date,   
substr(diagnosis_1,1,3) as diagnosis_1_substr,
substr(diagnosis_2,1,3) as diagnosis_2_substr,
substr(diagnosis_3,1,3) as diagnosis_3_substr,
from episode_ch_filtered
where episode_sequence = 1
order by person_id),
#up to this point reprents the code to identify and clean all seocndary inpatient episodes for indiivudals in the connected bradford cohort in the 12 months after admission,the next line of code will identify admission related to falls, will then union this with a table for falls in AE. admissions related to falls was defined as any admission with a SNOMED OR ICD10 code available. 

fall_ch as (
select
*,
row_number() over (partition by person_id order by tbl_episode_start_date) as admission_seq
from episode_ch_final 
where episode_ch_final.diagnosis_1_substr in(select Code from `yhcr-prd-bradfor-bia-core.CB_2172.falls__fragilityfracture_icd10`) OR episode_ch_final.diagnosis_2_substr in(select Code from `yhcr-prd-bradfor-bia-core.CB_2172.falls__fragilityfracture_icd10`) OR episode_ch_final.diagnosis_3_substr in(select Code from `yhcr-prd-bradfor-bia-core.CB_2172.falls__fragilityfracture_icd10`) or episode_ch_final.diagnosis_1_substr in(select cast(code as string) from `yhcr-prd-bradfor-bia-core.CB_2172.falls_snomed` ) or diagnosis_2_substr in(select cast(code as string) from `yhcr-prd-bradfor-bia-core.CB_2172.falls_snomed` ) or episode_ch_final.diagnosis_3_substr in(select cast(code as string) from `yhcr-prd-bradfor-bia-core.CB_2172.falls_snomed` )
order by person_id),
#this then filters for the first hopsital spell with a fall coded in any of the diagnosis codes. it is theoretically possible that a patient had an inpatient fall which was coded in the co-moribdities rather than the primary reason for presntation, but it is assumed the margin of error is relatively small.

fall_ch_inpatient as (
select
*,
from fall_ch
where admission_seq = 1
order by person_id)

select
person_id,
first_episodestartdate,
tbl_episode_start_date
from fall_ch_inpatient

union distinct

select
person_id,
first_episodestartdate,
tbl_ae_start_date
from fall_ae_final),

#then we union distint inpatient episodes and ED episodes together before adding to airedale

fall_union as (
select 
person_id,
first_episodestartdate,
tbl_episode_start_date
from fall_bradford


union all 

select
person_id,
fall_aire.first_episodestartdate,
fall_aire.episode_start
from
fall_aire ),

sequence as (
select
*,
row_number() over (partition by person_id order by tbl_episode_start_date) admission_sequence
from fall_union
order by person_id)

select
*
from sequence 
where admission_sequence = 1
order by person_id)
